In [ ]:
import numpy as np
import rasterio
import joblib
import os
import matplotlib.pyplot as plt

# Load trained model
model = joblib.load('../models/random_forest_lulc.pkl')

# Load composite image
with rasterio.open('../data/raw/barishal_composite_2022.tif') as src:
    composite = src.read().astype(np.float32)  # shape: (bands, height, width)
    profile = src.profile

# Prepare data for prediction
bands, height, width = composite.shape

# Reshape composite to (num_pixels, num_features)
X_full = composite.reshape(bands, height * width).T  # shape: (pixels, bands)

# Predict land cover classes for all pixels
y_pred = model.predict(X_full)  # shape: (pixels,)

# Reshape prediction back to raster dimensions
classification_map = y_pred.reshape(height, width).astype(np.uint8)

# Save the classification map as GeoTIFF
output_path = '../results/barishal_lulc_map.tif'

# Update metadata for single band uint8 output
profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(classification_map, 1)

print(f"Classification map saved to {output_path}")

# Visualise
plt.figure(figsize=(10, 8))
plt.title('Predicted Land Cover Map')
plt.imshow(classification_map, cmap='tab10')
plt.colorbar(ticks=range(5), label='Class')
plt.show()